In [27]:
from transformers import pipeline
import pandas as pd

In [28]:
qa_model = pipeline(task='table-question-answering',
                    model='dsba-lab/koreapas-finetuned-korwikitq')

# 답변용 데이터 불러오기
df = pd.read_csv('./pokemon_combined.csv')

c:\Users\user\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [29]:
df

,Name,Type,Species,Height,Weight,Abilities,Catch rate,Base Friendship,Base Exp.,Growth Rate,Gender,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,Bulbasaur,Grass Poison,Seed Pokémon,0.7,6.9,1. OvergrowChlorophyll (hidden ability),45,50,64,Medium Slow,"87.5% male, 12.5% female",45,49,49,65,65,45
1,Ivysaur,Grass Poison,Seed Pokémon,1.0,13.0,1. OvergrowChlorophyll (hidden ability),45,50,142,Medium Slow,"87.5% male, 12.5% female",60,62,63,80,80,60
2,Venusaur,Grass Poison,Seed Pokémon,2.4,155.5,Thick Fat,45,50,281,Medium Slow,"87.5% male, 12.5% female",80,100,123,122,120,80
3,Charmander,Fire,Lizard Pokémon,0.6,8.5,1. BlazeSolar Power (hidden ability),45,50,62,Medium Slow,"87.5% male, 12.5% female",39,52,43,60,50,65
4,Charmeleon,Fire,Flame Pokémon,1.1,19.0,1. BlazeSolar Power (hidden ability),45,50,142,Medium Slow,"87.5% male, 12.5% female",58,64,58,80,65,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1009,Raging Bolt,Electric Dragon,Paradox Pokémon,5.2,480.0,1. Protosynthesis,10,0,0,Slow,Genderless,125,73,91,137,89,75
1010,Iron Boulder,Rock Psychic,Paradox Pokémon,1.5,162.5,1. Quark Drive,10,0,0,Slow,Genderless,90,120,80,68,108,124
1011,Iron Crown,Steel Psychic,Paradox Pokémon,1.6,156.0,1. Quark Drive,10,0,0,Slow,Genderless,90,72,100,122,108,98
1012,Terapagos,Normal,Tera Pokémon,1.7,77.0,Teraform Zero,255,0,0,Slow,"50% male, 50% female",160,105,110,130,110,85


In [56]:
df = df.iloc[:,:3]
# df[['Type1', 'Type2']] = df['Type'].str.split(' ', expand=True)
# df.drop('Type', axis=1, inplace=True)
df.rename({'Name' : '이름', 'Type' : '타입','Species' : '종'}, axis=1, inplace=True)

In [60]:
df.iloc[39]

이름               Paras
타입           Bug Grass
종     Mushroom Pokémon
Name: 39, dtype: object

In [58]:
qa_model(query='Bulbasaur의 종은 뭐야?', table=df)

c:\Users\user\anaconda3\Lib\site-packages\transformers\models\tapas\tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
c:\Users\user\anaconda3\Lib\site-packages\transformers\models\tapas\tokenization_tapas.py:1472: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]


{'answer': 'Seed Pokémon, Mushroom Pokémon',
 'coordinates': [(0, 2), (39, 2)],
 'cells': ['Seed Pokémon', 'Mushroom Pokémon']}

In [61]:
def make_answer(q):
    answer = qa_model(query = q, table = df)

    if answer['answer'] == '':
        return '질문의 답을 찾지 못했습니다.'
    
    elif answer['coordinates'][0][-1] == 1: # 질문에 대한 답이 첫 번째 열(0번 열 : Error 열)일 경우
        answer2 = answer['answer'] # 정답 저장.

        return f'''그 포켓몬의 타입은 {answer2} 입니다.'''
    
    elif answer['coordinates'][0][-1] == 2: # 질문에 대한 답이 첫 번째 열(0번 열 : Error 열)일 경우
        answer2 = answer['cells'][0]

        return f'''그 포켓몬의 종은 {answer2} 입니다.'''
    
    else: # 질문이 설명인 경우.
        answer2 = answer['cells'][0][0] # 

        return f'''그런 특징을 가진 포켓몬 중 하나는 {answer2}입니다.'''

In [49]:
inputs = input('질문이나 에러 메시지를 입력해 주세요: ')

In [62]:
return_msg = make_answer(inputs)

c:\Users\user\anaconda3\Lib\site-packages\transformers\models\tapas\tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
c:\Users\user\anaconda3\Lib\site-packages\transformers\models\tapas\tokenization_tapas.py:1472: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]


In [63]:
print(return_msg)

그 포켓몬의 종은 Seed Pokémon 입니다.
